In [11]:
import pandas as pd
import numpy as np
import os 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import classification_report
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
# thursday = pd.read_csv('Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',low_memory = False)
# thursday = thursday.rename(str.lstrip, axis='columns')

In [3]:
# # data preprocessing
# os.chdir(r"C:\Users\Kael\Desktop\MachineLearningCVE-20230501T005316Z-001\MachineLearningCVE")
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# nRowsRead = None 
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# nRowsRead = None 

# df6 = pd.read_csv("Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")

# df = pd.concat([df6,df6])
# del df6

# nRow, nCol = df.shape

In [4]:
df = pd.read_csv('Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',low_memory = False)

In [5]:
#change the variable types for low memory usage
#int64 to int32,,, float64 to float32
integer = []
f = []
for i in df.columns[:-1]:
    if df[i].dtype == "int64": integer.append(i)
    else : f.append(i)

df[integer] = df[integer].astype("int32")
df[f] = df[f].astype("float32")

In [6]:
df.columns =  df.columns.str.strip()
# drop nan and infinite rows
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)]
# merging similar classes with low instances
df["Label"] = df["Label"].replace(["Web Attack: Brute Force","Web Attack: XSS","Web Attack: Sql Injection"],"Web Attack")
# drop duplicate rows
df =  df.drop_duplicates(keep="first")
df.reset_index(drop=True,inplace=True)

<ipython-input-6-e62fcfcd661e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Label"] = df["Label"].replace(["Web Attack: Brute Force","Web Attack: XSS","Web Attack: Sql Injection"],"Web Attack")


In [7]:
#feature reduction 
#dropping very high correlated features 
numeric_columns = df.select_dtypes(include=[np.number])

corr_matrix = numeric_columns.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features 
df =  df.drop(to_drop, axis=1)
df.shape

##############################################################################

<ipython-input-7-563e00f0e2d2>:8: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


(164179, 54)

In [18]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df.head(1)

,Destination Port,Flow Duration,Total Fwd Packets,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Mean,Fwd IAT Std,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Variance,FIN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,Down/Up Ratio,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Init_Win_bytes_forward,Init_Win_bytes_backward,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Std,Label
0,389,113095465,48,9668,403,0,201.416672,203.548294,923,316,417.166656,231.080948,174.012283,0.63663,1592893.875,4597264.5,16500000,3,2406286.5,5491986.0,4917177.5,7084368.5,16500000,1,0,0,0,0.21221,0,923,275.109589,54677.988281,0,0,0,1,0,0,0,0,0,0,0,0,0,571,2079,32,203985.5,575837.25,1629110,379,4277541.0,BENIGN


In [9]:
features = df.iloc[:, :df.shape[1] - 1]
labels = df.iloc[:, df.shape[1] - 1:]
labels = labels.values.ravel()

# df = pd.DataFrame(features)
X_train, X_test, y_train, y_test = train_test_split(features, labels, train_size=0.8, test_size=0.2, stratify=labels)

In [16]:
model = LogisticRegression()
model.fit(X_train, y_train)

print("Score:", model.score(X_train, y_train))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Score: 0.9914346405975195


In [17]:
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                            precision    recall  f1-score   support

                    BENIGN       1.00      1.00      1.00     32407
  Web Attack � Brute Force       0.57      0.82      0.67       294
Web Attack � Sql Injection       0.00      0.00      0.00         4
          Web Attack � XSS       0.11      0.01      0.01       131

                  accuracy                           0.99     32836
                 macro avg       0.42      0.46      0.42     32836
              weighted avg       0.99      0.99      0.99     32836



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
